In [1]:
import math
import numpy as np
%pylab inline
%run Thermodatabase1.ipynb

Populating the interactive namespace from numpy and matplotlib
Populating the interactive namespace from numpy and matplotlib


In [2]:
# getKthermo           Calculates...
##                     Major cations - anions
##                     Minor cations - Major anions
##                     Minor anions -  Major cations
##                     DIVALENTS - Major anions
# ************************* ENTALPHY AND HEAT CAPACITY DATA *****************************
# 1) DH(25C) for H2O = H+ + OH-  is + 55.84 kJ/mol (Atkins ; Phys Chem. tables)
# 
# 2) Heat Capacity data (dh25)
#    HYDROLYSIS -> from Baes and Mesmer Am.j.Sci.(1981)  kcal/mol
# 
# 3) See Millero and Hawke 1992 Mar Chem 40 and refs within for;
#        Mn-OH(12),-HCO3,-CO3,-SO4,-Cl
#        Fe-OH(12),-HCO3,-CO3(12),-SO4,-Cl
#        Co-OH(12),-HCO3,-CO3,-SO4,-Cl
#        Ni-OH(12),-HCO3,-CO3,-SO4(12),-Cl
#        Cu(II)-OH(12),-HCO3,-CO3(12),-SO4,-Cl
#        Zn-OH(12),-HCO3,-CO3(12),-SO4(12),-Cl
#        Cd-OH(12),-HCO3,-CO3,-SO4,-Cl(123)
#        Pb-OH(12),-HCO3,-CO3(12),-SO4(12),-Cl(123)
# 
# 4) HS data from Zhang and Millero 1994
# 
# 5) FROM BYRNE ET AL. MAR CHEM. 1988
#       BeOH(123),BeF(123)
#       PbOH (12), PbCO3(12), PbCl(123), PbSO4(12)
#       MnCO3,MnOH(12),FeCO3,FeOH(12),CoCO3,CoOH(12)
#       NiCO3,NiOH(12)
#       BeF(12),BeOH(123)
#
#       5a)DELTA H DATA .
#               MULTISTEP HYDROLYSIS DELTAH CALC. FROM
#                   DELTA H = SUM OF STEP DELTA H  EX. DELTAH2 = -1.36 LOG K2 - 5.3 + 3.64(z-n+1)
#               
#  6) DELTA Cp IS A FUNCTION OF IONIC RADIUS (Baes and Mesmer 1981) from CRC Handbook 70th ed.
#     DELTA Cp data;CdOH(12),CoOH(12),CuOH(12),FeOH(12),MgOH,CaOH,MnOH(12),NiOH(12), ZnOH(12),BeOH(123),PbOH
##                           TRIVALENTS - Major anions
# ******************************************SOURCES**********************************************
# 
# Hydrolysis data log K11 from (Baes and Mesmer, 1976 book) ... actually from Millero 1992
# Other Ion pairs from Millero 1992 Geochim. Cosmochim. Acta 56, pp 3123-3132
# For Fe(III);  Hydrolysis=Millero Pierrot 2007 Geochimica et Cosmochimica Acta 71
#
# *************************Enthalpy and Heat Capacity data for Hydrolysis *****************************
#
# DH(25) for H2O = H+ + OH-  is + 55.84 kJ/mol (Atkins ; Phys Chem. tables)
# DELTA H DATA FROM BYRNE ET AL.(1988) MAR. CHEM.
# FIRST STEP HYDROLYSIS DELTAH CALC. FROM
# DELTA H = -0.836 LOG K11 + 5.70 (z/d)z = charge of cation (3), d = M-OH distance (Shannon and Prewitt 1969)
# d is calculated from ionic radii from CRC 70th ed. F-187 + 1.40 (for O2- radii - see CRC 70th B215)
# OTHER STEP HYDROLYSIS DELTAH CALC. ESTIMATED FROM (Baes and Mesmer 1981);
# DELTA H = -1.36 (LOG K1x-LOG K1(x-1)) - 5.3 + 3.64 *(z))*4184 - DH25H2O  + DELTA H (previous step)
# z=charge of M(OH) species + 1 in the stepex; Er3+ + 4 OH- <==> Er(OH)4-    z=0
#
# For Al3+, In3+ and Ga3+ ;
# Hydrolysis (Byrne et al. 1988 + correction to S=0 in paper)
# other than hydrolysis; Ref. = Turner, Whitfield, Dickson  Geochim. Cosmochim. Acta 45 pp855 (1981)
# Heat Capacity data from Baes and Mesmer Am.j.Sci.(1981) (only for Hydrolysis)
# DELTA Cp IS A FUNCTION OF IONIC RADIUS (Baes and Mesmer 1981)

## ---------------------------------------------------------------------------------------------
##  Assign values to global variables
mjCaMax = 5             #In python array goes 0,1,2,3,4,5,6, so mjCaMax must go to 7 to include the 6th variable
mnCaMax = 37
CaMax = mjCaMax + mnCaMax

mjAnMax = 7             #In python array goes 0,1,2,3,4,5,6,7,8 so mjCaMax must go to 8 to include the 7th variable
mnAnMax = 19
AnMax = mjAnMax + mnAnMax

NeutMax = 7

## ------------------------------------------------------------------------------

def Kthermo(Tc):
    
    maxr = size(Tc)       
    T = Tc + 273.15
    
    def Eq1_K(T,A):
        # For: The thermodynamic constants in Table I (MP98)
        lnKatT = A[0] + A[1]/T + A[2]*log(T) + A[3]*T
        KatT = exp(lnKatT)
        return KatT
    
    def Eq2_K(T,B):
        # For: The thermodynamic constants in Table II (MP98)
        pKatT = B[0]+ B[1]/T  + B[2]*T
        KatT = 10**(pKatT)
        return KatT
    
    def Eq3_K(T,C):
        # Analytic expression in PhreeqC. Takes precedence of vanthoff if defined
        logKatT = C[0]+ C[1]*T + C[2]/T  + C[3]*log10(T) + C[4]/(T**2)  + C[5]*(T**2)
        KatT = 10**(logKatT)
        return KatT
    
    def VH1(T1,D):
        # Using van't hoff equation. Assuming enthalpy change is constant w.r.t to change in T
        KT1overKT2 = exp((D[1]/8.314)*(1/298.15 - 1/T1))
        KT1 = 10**D[0] * KT1overKT2
        return KT1
    
    def VH2(LogK25, DeltaH25, DeltaCp, T):
        # Using van't hoff equation. Accounting for enthalpy change w.r.t T using delta C.P where possible
        DeltaHT = DeltaH25 + DeltaCp * (T - 298.15)
        logK = LogK25 + ((DeltaHT / 8.314) * (1 / 298.15 - 1 / T))
        KatT = 10**(logK)
        return KatT
    
    # expect 1 input value for  T
    def Cp1 (T):
        tt =sorted([298.15,333.15,373.15,423.15,473.15])  #sorted() makes sure list is always in order
        dcp_h2o = [-55,-47,-44,-45,-51]
        #by checking range(len(tt)) you know you will always be iterating each value
        for i in range(len(tt)):
            if T <= tt[i]:      
                Dcp = -dcp_h2o[i] * 4.184
                return Dcp
        if T>tt[i]:  #no need to specify larger then, there is no other option
            Dcp = -dcp_h2o[-1] * 4.184      #[-1] takes the last value of the dcp_h2o array
        return Dcp
        
    def Cp2 (R,T):
        # Whilst cp data extends to 200, the model is only consistently valid between 0 - 50
        tt = sorted([298.15, 333.15, 373.15, 423.15, 473.15])  #sorted() makes sure list is always in order
        dcp_h2o = [-55,-47,-44,-45,-51]
        r = [0.5, 0.8, 1.1]
        ddcp = [[-15, -11, -7],[-14, -11, -9],[-10, -7, -6],[-10, -7, -4],[-10, -6, -4]]
        
        for i in range(len(tt)):
            if T <= tt[i]:
                if R <= r[1]:
                    Dcp = (ddcp[i][0] + ((ddcp[i][1] - ddcp[i][0]) / (r[1] - r[0])) * (R - r[0]) - dcp_h2o[i]) * 4.184
                else: 
                    Dcp = (ddcp[i][1] + ((ddcp[i][2] - ddcp[i][1]) / (r[2] - r[1])) * (R - r[1]) - dcp_h2o[i]) * 4.184
                return Dcp
                           
            if T>tt[-1]:          #model at 200
                Dcp = (ddcp[4][1] + (ddcp[4][2] - ddcp[4][1]) / (r[2] - r[1]) * (R - r[1]) - dcp_h2o[-1]) * 4.184
                return Dcp
    
    ## ------------------------------------------------------------------------------
    
    mjKthermo     = np.zeros([5,8])
    mnCat_Kthermo = np.zeros([3,7,7])
    divKthermo    = np.zeros([4,10,6]) 
    trivKthermo   = np.zeros([4,20,6])
    
    # VAR = [Tc]

    # for i in range(len(VAR)):
    #   Y1 = mjCaMax * [i]
    #   Y2 = 7  *[i]
    #   Y3 = 10 *[i] 
    #   Y4 = 20 *[i]
    
    Tx = T
        
    ## Major Cat - An: Association constants
        
    ## Na
    mjKthermo [0,0]  = Eq3_K(Tx,T_param_NaCl)      # NaCl
    mjKthermo [0,1]  = VH1(Tx,T_logk25NaSO4)       # NaSO4   
    mjKthermo [0,2]  = VH1(Tx,T_logk25NaCO3)       # NaCO3    
    mjKthermo [0,3]  = VH1(Tx,T_logk25NaHCO3)      # NaHCO3    Eq3_K(T,param_NaHCO3)
    mjKthermo [0,4]  = Eq3_K(Tx,T_param_NaBr)      # NaBr
    mjKthermo [0,5]  = Eq3_K(Tx,T_param_NaF)       # NaF 
    mjKthermo [0,6]  = T_k25NaB                    # NaB(OH)4  
    mjKthermo [0,7]  = T_k25NaOH                   # NaOH 
        
    ## K
    mjKthermo [1,0]  = Eq3_K(Tx,T_param_KCl )      # KCl  
    mjKthermo [1,1]  = Eq3_K(Tx,T_param_KSO4)      # KSO4
    mjKthermo [1,4]  = Eq3_K(Tx,T_param_KBr)       # KBr
    # mjKthermo (2+Y1[0],8) = T_k25KOH             # KOH 
        
    ##Mg
    mjKthermo [2,0]  = Eq3_K(Tx,T_param_MgCl)      # MgCl
    mjKthermo [2,1]  = Eq3_K(Tx,T_param_MgSO4)     # MgSO4                      
    mjKthermo [2,2]  = Eq3_K(Tx,T_param2_MgCO3)    # MgCO3
    mjKthermo [2,3]  = Eq3_K(Tx,T_param_MgHCO3)    # MgHCO3
    mjKthermo [2,5]  = VH1(Tx, T_k25MgF)           # MgF       Eq3_K(T,param2_MgF)   
    mjKthermo [2,6]  = T_k25MgB                    # MgB(OH)4    
    mjKthermo [2,7]  = Eq2_K(Tx,T_param_MgOH)      # MgOH  
        
    ## Ca
    mjKthermo [3,0]  = Eq3_K(Tx,T_param_CaCl)      # CaCl
    mjKthermo [3,1]  = VH1(Tx,T_k25CaSO4)          # CaSO4    Eq3_K(T,param_CaSO4); 
    mjKthermo [3,2]  = Eq3_K(Tx,T_param2_CaCO3)    # CaCO3      
    mjKthermo [3,3]  = Eq3_K(Tx,T_param_CaHCO3)    # CaHCO3
    mjKthermo [3,5]  = VH1(Tx,T_k25CaF)            # CaF      Eq3_K(T,param2_CaF)
    mjKthermo [3,6]  = T_k25CaB                    # CaB(OH)4 
    mjKthermo [3,7]  = T_k25CaOH                   # CaOH 
        
    ## Sr
    mjKthermo [4,0]  = Eq3_K(Tx,T_param_SrCl)      # SrCl
    mjKthermo [4,1]  = VH1(Tx,T_logk25SrSO4)       # SrSO4   
    mjKthermo [4,2]  = Eq2_K(Tx,T_param_SrCO3)     # SrCO3
    mjKthermo [4,3]  = Eq3_K(Tx,T_param_SrHCO3)    # SrHCO3
    mjKthermo [4,5]  = Eq3_K(Tx,T_param_SrF)       # SrF
    mjKthermo [4,6]  = T_k25SrB                    # SrB(OH)4 
    mjKthermo [4,7]  = T_k25SrOH                   # SrOH 
        
    ## ************************** Minor Cat - Major An  *************************
    #  lig/An
    #  1. Cl      
    #  2. SO4    
    #  3. CO3    
    #  4. HCO3 
    #  5. Br
    #  6. F       
    #  7. OH   
    
    # matlab = [row, integer along, set]
    # python = [set, row, integer along] (keep in mind python starts at 0)

    ## H
    mnCat_Kthermo[0,0,1] = Eq1_K(Tx, T_param_HSO4)  # HSO4
    mnCat_Kthermo[0,0,5] = Eq1_K(Tx, T_param_HF)    # HF
        
    ## Li
    mnCat_Kthermo[0,1,0] = Eq3_K(Tx,T_param_LiCl)   # LiCl
    mnCat_Kthermo[0,1,1] = T_k25LiSO4               # LiSO4
    mnCat_Kthermo[0,1,5] = VH1(Tx, T_param_LiF)     # LiF
    # mnCat_Kthermo(2+Y2[0],7,1) = k25LiOH        # LiOH
        
    ## Rb 
    mnCat_Kthermo[0,2,0] = Eq3_K(Tx, T_param_RbCl)  # RbCl
    mnCat_Kthermo[0,2,1] = T_k25RbSO4               # RbSO4
    mnCat_Kthermo[0,2,4] = Eq3_K(Tx,T_param_RbBr)   # RbBr
    mnCat_Kthermo[0,2,5] = Eq3_K(Tx,T_param_RbF)    # RbF 
        
    ## Cs
    mnCat_Kthermo[0,3,0] =  Eq3_K(Tx,T_k25CsCl)     # CsCl
    mnCat_Kthermo[0,3,1] =  T_k25CsSO4              # CsSO4  
    mnCat_Kthermo[0,3,2] =  T_k25CsCO3              # CsCO3
    mnCat_Kthermo[0,3,4] =  Eq3_K(Tx,T_k25CsBr)     # k25CsBr
        
    ## NH4
    mnCat_Kthermo[0,4,1] =  T_k25NH4SO4             # NH4SO4
        
    ## Ba
    mnCat_Kthermo[0,5,0] = Eq3_K(Tx,T_param_BaCl)   # BaCl
    mnCat_Kthermo[0,5,1] = T_k25BaSO4               # BaSO4
    mnCat_Kthermo[0,5,2] = Eq3_K(Tx,T_param_BaCO3)  # BaCO3   
    mnCat_Kthermo[0,5,3] = Eq3_K(Tx,T_param_BaHCO3) # BaHCO3
    mnCat_Kthermo[0,5,5] = Eq3_K(Tx,T_param_BaF)    # BaF
        
    ## Cu(I)
        
    # Cl
    logk25 = 3.1 
    dh25 = 0 * 4184 
    dcp = 0
    mnCat_Kthermo[0,6,0] = VH2(logk25, dh25, dcp, Tx) 
        
    # Cl2
    logk25 = 5.42 
    dh25 = 0 * 4184 
    dcp = 0
    mnCat_Kthermo[1,6,0] = VH2(logk25, dh25, dcp, Tx) 
        
    # Cl3
    logk25 = 4.75 
    dh25 = 0 * 4184 
    dcp = 0
    mnCat_Kthermo[2,6,0] = VH2(logk25, dh25, dcp, Tx) 
        
    ## DIVALENTS AND TRIVALENTS
    # Using Vant Hoff equation 
    # ln k(T) = ln k(25) +(DELTAH(T)/R)(1/298.15 - 1/T)  

    # Using Kirchoffs equation to accounting for changes in delta H with T using heat capacity data: 
    # DELTAH(T) = DELTAH(298.15) + DELTACp (T-298.15)   

    ##   DH(25C) AND Kw for H2O (to convert hydrolysis constants to stability constants)
    DH25H2O = 55840                     # DH(25C) for H2O = H+ + OH-  is + 55.84 kJ/mol
    pKw = 13.995
        
    ## ************************** DISSOCIATION CONSTANTS FOR DIVALENTS *************************
    # I = CATION  J = ligand  K = number of ligands
    #  lig/An
    
    #  0. Cl      K = 1, 2, 3, 4  
    #  1. SO4     K = 1, 2 
    #  2. CO3     K = 1, 2, 3
    #  3. HCO3    K = 1
    #  4. F       K = 1, 2, 3, 4
    #  5. OH      K = 1, 2, 3, 4

    ## Mn - 12
    # Cl
    logk25 = 0.39 
    dh25 = 0 
    dcp = 0
    divKthermo[0,0,0] = VH2(logk25, dh25, dcp, Tx)
        
    # SO4
    logk25 = 2.26
    dh25 = 0
    dcp = 0                  
    divKthermo[0,0,1]  = VH2(logk25, dh25, dcp, Tx)
        
    # CO3
    logk25 = 4.1
    dh25 = 3 * 4184
    dcp = 0
    divKthermo[0,0,2] = VH2(logk25, dh25, dcp, Tx)
        
    # HCO3
    logk25 = 1.28
    dh25 = 0
    dcp = 0                  
    divKthermo[0,0,3]  = VH2(logk25, dh25, dcp,Tx)
        
    # OH (dh25: 15 - 42)
    logk25 = 3.4
    dh25 = (14.4 * 4184 - DH25H2O)
    dcp = Cp2(0.8, Tx)
    divKthermo[0,0,5]  = VH2(logk25, dh25, dcp,Tx)
        
    # OH2
    logk25 = 5.78
    dh25 = (14.4 + (-1.36) * (logk25 - log(divKthermo[0,0,5]) - pKw) - 5.3 + 3.64 * (2 - 2 + 1)) * 4184 - 2 * DH25H2O
    dcp = Cp1(Tx)
    divKthermo[1,0,5] = VH2(logk25, dh25, dcp,Tx)

    ## Fe(II) - 13
    # Cl
    logk25 = 0.32
    dh25 = 0
    dcp = 0
    divKthermo[0,1,0] = VH2(logk25, dh25, dcp,Tx) 
        
    # SO4
    logk25 = 2.2
    dh25 = 0
    dcp = 0
    divKthermo[0,1,1] = VH2(logk25, dh25, dcp,Tx) 
        
    # CO3
    logk25 = 5.45
    dh25 = 3 * 4184
    dcp = 0
    divKthermo[0,1,2] = VH2(logk25, dh25, dcp,Tx) 
        
    # CO32
    logk25 = 7.17
    dh25 = 0
    dcp = 0
    divKthermo[1,1,2] = VH2(logk25, dh25, dcp,Tx) 
        
    # HCO3
    logk25 = 1.47
    dh25 = 0
    dcp = 0
    divKthermo[0,1,3] = VH2(logk25, dh25, dcp,Tx) 
        
    # OH (dh25: 13.2 25 - 300; 12.7 150 -330)
    logk25 = 4.49
    dh25 = 13.2 * 4184 - DH25H2O
    dcp = Cp2(0.74, Tx)
    divKthermo[0,1,5] = VH2(logk25, dh25, dcp,Tx) 
        
    # OH2
    logk25 = 7.39
    dh25 = (13.2 + (-1.36) * (logk25 - log(divKthermo[0,1,5]) - pKw) - 5.3 + 3.64 * (2 - 2 + 1)) * 4184 - 2 * DH25H2O
    dcp = Cp1(Tx)
    divKthermo[1,1,5] = VH2(logk25, dh25, dcp,Tx) 
        
    ## Co - 14
    # Cl
    logk25 = 0.26
    dh25 = 0
    dcp = 0
    divKthermo[0,2,0] = VH2(logk25, dh25, dcp, Tx)
        
    # SO4
    logk25 = 3.06
    dh25 = 0
    dcp = 0
    divKthermo[0,2,1] = VH2(logk25, dh25, dcp, Tx)
        
    # CO3
    logk25 = 4.49
    dh25 = 3 * 4184
    dcp = 0
    divKthermo[0,2,2] = VH2(logk25, dh25, dcp, Tx)
        
    # HCO3
    logk25 = 1.41
    dh25 = 0
    dcp = 0
    divKthermo[0,2,3] = VH2(logk25, dh25, dcp, Tx) 
        
    # OH (dh25: 25 -200)
    logk25 = 4.34
    dh25 = 14.6 * 4184 - DH25H2O
    dcp = Cp2(0.72, Tx)
    divKthermo[0,2,5] = VH2(logk25, dh25, dcp, Tx)
        
    # OH2
    logk25 = 9.19
    dh25 = (14.6 + (-1.36) * (logk25 - log(divKthermo[0,2,5]) - pKw) - 5.3 + 3.64 * (2 - 2 + 1)) * 4184 - 2 * DH25H2O
    dcp = Cp1(Tx)
    divKthermo[1,2,5] = VH2(logk25, dh25, dcp, Tx)

    # Ni -15
    # Cl
    logk25 = 0.17
    dh25 = 0
    dcp = 0
    divKthermo[0,3,0] = VH2(logk25, dh25, dcp, Tx)
        
    # SO4
    logk25 = 2.29
    dh25 = 0
    dcp = 0
    divKthermo[0,3,1] = VH2(logk25, dh25, dcp, Tx) 
        
    # SO42
    logk25 = 3.2
    dh25 = 0
    dcp = 0
    divKthermo[1,3,1] = VH2(logk25, dh25, dcp, Tx) 
        
    # CO3
    logk25 = 5.37
    dh25 = 3 * 4184
    dcp = 0
    divKthermo[0,3,2] = VH2(logk25, dh25, dcp, Tx)
        
    # HCO3
    logk25 = 1.63
    dh25 = 0
    dcp = 0
    divKthermo[0,3,3] = VH2(logk25, dh25, dcp, Tx)
        
    # OH dh25: 15-300
    logk25 = 4.13
    dh25 = 11.9 * 4184 - DH25H2O
    dcp = Cp2(0.69, Tx)
    divKthermo[0,3,5] = VH2(logk25, dh25, dcp, Tx) 
        
    # OH2
    logk25 = 8.99
    dh25 = (11.9 + (-1.36) * (logk25 - log(divKthermo[0,3,5]) - pKw) - 5.3 + 3.64 * (2 - 2 + 1)) * 4184 - 2 * DH25H2O
    dcp = Cp1(Tx)
    divKthermo[1,3,5] = VH2(logk25, dh25, dcp, Tx)
        
    ## Cu(II) - 16 
    # Cl
    logk25 = 0.53
    dh25 = 0
    dcp = 0
    divKthermo[0,4,0] = VH2(logk25, dh25, dcp, Tx)
        
    # SO4
    logk25 = 2.36
    dh25 = 1.9 * 4184
    dcp = 0
    divKthermo[0,4,1] = VH2(logk25, dh25, dcp, Tx)
        
    # CO3
    logk25 = 6.73
    dh25 = 3 * 4184
    dcp = 0
    divKthermo[0,4,2] = VH2(logk25, dh25, dcp, Tx)
        
    # CO32
    logk25 = 10.41
    dh25 = 0
    dcp = 0
    divKthermo[1,4,2] = VH2(logk25, dh25, dcp, Tx)
        
    # HCO3
    logk25 = 1.82
    dh25 = 0
    dcp = 0
    divKthermo[0,4,3] = VH2(logk25, dh25, dcp, Tx)
        
    # OH
    logk25 = 6
    dh25 = 12 * 4184 - DH25H2O
    dcp = Cp2(0.72, Tx)
    divKthermo[0,4,5] = VH2(logk25, dh25, dcp, Tx)
        
    # OH2
    logk25 = 11.74
    dh25 = (12 + (-1.36) * (logk25 - log(divKthermo[0,4,5]) - pKw) - 5.3 + 3.64 * (2 - 2 + 1)) * 4184 - 2 * DH25H2O
    dcp = Cp1(Tx)
    divKthermo[1,4,5] = VH2(logk25, dh25, dcp, Tx)

    ## Zn -17
    # Cl
    logk25 = 0.33
    dh25 = 0
    dcp = 0
    divKthermo[0,5,0] = VH2(logk25, dh25, dcp, Tx)
        
    # SO4
    logk25 = 2.36
    dh25 = 0
    dcp = 0
    divKthermo[0,5,1] = VH2(logk25, dh25, dcp, Tx)
        
    # SO42
    logk25 = 3.63
    dh25 = 0
    dcp = 0
    divKthermo[1,5,1] = VH2(logk25, dh25, dcp, Tx)
        
    # CO3
    logk25 = 4.71
    dh25 = 3 * 4184
    dcp = 0
    divKthermo[0,5,2] = VH2(logk25, dh25, dcp, Tx)
        
    # CO32
    logk25 = 7.25
    dh25 = 0
    dcp = 0
    divKthermo[1,5,2] = VH2(logk25, dh25, dcp, Tx)
        
    # HCO3
    logk25 = 1.74
    dh25 = 0
    dcp = 0
    divKthermo[0,5,3] = VH2(logk25, dh25, dcp, Tx)
        
    # OH dh25: 15-42
    logk25 = 5.03
    dh25 = 13.4 * 4184 - DH25H2O
    dcp = Cp2(0.74, Tx)
    divKthermo[0,5,5] = VH2(logk25, dh25, dcp, Tx)
        
    # OH2
    logk25 = 11.09
    dh25 = (13.4 + (-1.36) * (logk25 - log(divKthermo[0,5,5])- pKw) - 5.3 + 3.64 * (2 - 2 + 1)) * 4184 - 2 * DH25H2O
    dcp = Cp1(Tx)
    divKthermo[1,5,5] = VH2(logk25, dh25, dcp, Tx)
        
    ## UO2 -18
    # CO3
    logk25 = 10.8
    dh25 = 0
    dcp = 0
    divKthermo[0,6,2] = VH2(logk25, dh25, dcp, Tx)
        
    # CO32
    logk25 = 18.14
    dh25 = 3 * 4184
    dcp = 0
    divKthermo[1,6,2] = VH2(logk25, dh25, dcp, Tx)
        
    # CO33
    logk25 = 20.45
    dh25 = 0
    dcp = 0
    divKthermo[2,6,2] = VH2(logk25, dh25, dcp, Tx)
        
    # F
    logk25 = 6.8
    dh25 = 0
    dcp = 0
    divKthermo[0,6,4] = VH2(logk25, dh25, dcp, Tx)
        
    # F2
    logk25 = 9
    dh25 = 0
    dcp = 0
    divKthermo[1,6,4] = VH2(logk25, dh25, dcp, Tx)
        
    # F3
    logk25 = 11.3
    dh25 = 0
    dcp = 0
    divKthermo[2,6,4] = VH2(logk25, dh25, dcp, Tx)
        
    # F4
    logk25 = 12.6
    dh25 = 0
    dcp = 0
    divKthermo[3,6,4] = VH2(logk25, dh25, dcp, Tx)
        
    # OH
    logk25 = 8.12
    dh25 = 0
    dcp = Cp1(Tx)
    divKthermo[0,6,5] = VH2(logk25, dh25, dcp, Tx)
        
    ## Be -19 
    # CO3
    logk25 = 7.53
    dh25 = 0
    dcp = 0
    divKthermo[0,7,2] = VH2(logk25, dh25, dcp, Tx)
        
    # F
    logk25 = 5.61
    dh25 = 0
    dcp = 0
    divKthermo[0,7,4] = VH2(logk25, dh25, dcp, Tx)
        
    # F2
    logk25 = 9.68
    dh25 = -1 * 4184
    dcp = 0
    divKthermo[1,7,4] = VH2(logk25, dh25, dcp, Tx)
        
    # OH
    logk25 = 8.600001
    dh25 = 11.4 * 4184 - DH25H2O
    dcp = Cp2(0.35, Tx)
    divKthermo[0,7,5] = VH2(logk25, dh25, dcp, Tx)
        
    # OH2
    logk25 = 14.34
    dh25 = 21 * 4184 - 2 * DH25H2O
    dcp = Cp1(Tx)
    divKthermo[1,7,5] = VH2(logk25, dh25, dcp, Tx)
        
    # OH3
    logk25 = 18.74
    dh25 = 28.8 * 4184 - 3 * DH25H2O
    dcp = 0
    divKthermo[2,7,5] = VH2(logk25, dh25, dcp, Tx)
        
    # OH4
    logk25 = 18.57
    dh25 = 0
    dcp = 0
    divKthermo[3,7,5] = VH2(logk25, dh25, dcp, Tx)
        
    ## Cd -20
    # Cl
    logk25 = 1.56
    dh25 = 0.3 * 4184
    dcp = 0
    divKthermo[0,8,0] = VH2(logk25, dh25, dcp, Tx)
        
    # Cl2
    logk25 = 1.88
    dh25 = 0.9 * 4184
    dcp = 0
    divKthermo[1,8,0] = VH2(logk25, dh25, dcp,Tx)
        
    # Cl3 
    logk25 = 1.51
    dh25 = 2.4 * 4184
    dcp = 0
    divKthermo[2,8,0] = VH2(logk25, dh25, dcp, Tx)
        
    # SO4
    logk25 = 2.45
    dh25 = 0
    dcp = 0
    divKthermo[0,8,1] = VH2(logk25, dh25, dcp, Tx)
        
    # CO3
    logk25 = 4.35
    dh25 = 0
    dcp = 0
    divKthermo[0,8,2] = VH2(logk25, dh25, dcp, Tx)
        
    # HCO3
    logk25 = 1.37
    dh25 = 0
    dcp = 0
    divKthermo[0,8,3] = VH2(logk25, dh25, dcp, Tx)
        
    # OH DH25: 13.1 NO Tx RANGE; 12.5 60-100
    logk25 = 3.91
    dh25 = 13.1 * 4184 - DH25H2O
    dcp = Cp2(0.97, Tx)
    divKthermo[0,8,5] = VH2(logk25, dh25, dcp, Tx)
        
    # OH2
    logk25 = 7.63
    dh25 = (13.1 + (-1.36) * (logk25 - log(divKthermo[0,8,5]) - pKw) - 5.3 + 3.64 * (2 - 2 + 1)) * 4184 - 2 * DH25H2O
    dcp = Cp1(Tx) 
    divKthermo[1,8,5] = VH2(logk25, dh25, dcp, Tx) 

    ## Pb -21
    # Cl
    logk25 = 1.48
    dh25 = 4.4 * 4184
    dcp = 0
    divKthermo[0,9,0] = VH2(logk25, dh25, dcp, Tx)
        
    # Cl2
    logk25 = 2.03
    dh25 = 0 * 4184
    dcp = 0
    divKthermo[1,9,0] = VH2(logk25, dh25, dcp, Tx)
        
    # Cl3
    logk25 = 1.86
    dh25 = 0 * 4184
    dcp = 0
    divKthermo[2,9,0] = VH2(logk25, dh25, dcp, Tx)
        
    # SO4
    logk25 = 2.75
    dh25 = 0
    dcp = 0
    divKthermo[0,9,1] = VH2(logk25, dh25, dcp, Tx)
        
    # SO42
    logk25 = 4.51
    dh25 = 0
    dcp = 0
    divKthermo[1,9,1] = VH2(logk25, dh25, dcp, Tx)
        
    # CO3
    logk25 = 7
    dh25 = 3 * 4184
    dcp = 0
    divKthermo[0,9,2] = VH2(logk25, dh25, dcp, Tx)
        
    # CO32
    logk25 = 10.29
    dh25 = 0 * 4184
    dcp = 0
    divKthermo[1,9,2] = VH2(logk25, dh25, dcp, Tx)
        
    # HCO3
    logk25 = 2.05
    dh25 = 0
    dcp = 0
    divKthermo[0,9,3] = VH2(logk25, dh25, dcp, Tx)
        
    # OH
    logk25 = 6.28
    dh25 = 10.7 * 4184 - DH25H2O
    dcp = Cp2(1.2, Tx)
    divKthermo[0,9,5] = VH2(logk25, dh25, dcp, Tx)
        
    # OH2
    logk25 = 10.87
    dh25 = (10.7 + (-1.36) * (logk25 - log(divKthermo[0,9,5]) - pKw) - 5.3 + 3.64 * (2 - 2 + 1)) * 4184 - 2 * DH25H2O
    dcp = Cp1(Tx)
    divKthermo[1,9,5] = VH2(logk25, dh25, dcp, Tx)
        
    ## ************************** ASSOCIATION CONSTANTS FOR TRIVALENTS *************************
    # I = CATION  J = ligand  K = number of ligand
    #  lig 
    
    #  0. Cl      K = 1, 2, 3  
    #  1. SO4     K = 1, 2, 3  
    #  2. CO3     K = 1, 2
    #  3. HCO3    K = 1
    #  4. F       K = 1, 2, 3, 4
    #  5. OH      K = 1, 2, 3, 4

    ## La
    # Cl
    logk25 = 0.29
    dh25 = 0
    dcp = 0 
    trivKthermo [0,0,0] = VH2(logk25, dh25, dcp, Tx)
        
    # SO4
    logk25 = 3.21 
    dh25 = 0
    dcp = 0 
    trivKthermo [0,0,1] = VH2(logk25, dh25, dcp, Tx)
        
    # CO3
    logk25 = 6.82
    dh25 = 0
    dcp = 0 
    trivKthermo [0,0,2] = VH2(logk25, dh25, dcp, Tx)
        
    # CO32
    logk25 = 11.31
    dh25 = 0
    dcp = 0
    trivKthermo [1,0,2] = VH2(logk25, dh25, dcp, Tx)
        
    # HCO3
    logk25 = 2.02
    dh25 = 0
    dcp = 0 
    trivKthermo [0,0,3] = VH2(logk25, dh25, dcp, Tx)
        
    # F
    logk25 = 3.12
    dh25 = 0
    dcp = 0 
    trivKthermo [0,0,4] = VH2(logk25, dh25, dcp, Tx)
        
    # OH
    logk25 = 5.1
    rad = 1.061 + 1.4
    dh25 = (-0.836 * logk25 + 5.7 * (3 / rad)) * 4184 - DH25H2O
    dcp = Cp2(rad,Tx) 
    trivKthermo [0,0,5] = VH2(logk25, dh25, dcp, Tx)

    ## Ce
    # Cl
    logk25 = 0.31
    dh25 = 0
    dcp = 0 
    trivKthermo [0,1,0] = VH2(logk25, dh25, dcp, Tx)
        
    # SO4
    logk25 = 3.29
    dh25 = 0
    dcp = 0 
    trivKthermo [0,1,1] = VH2(logk25, dh25, dcp, Tx)
        
    # CO3
    logk25 = 6.95
    dh25 = 0
    dcp = 0 
    trivKthermo [0,1,2] = VH2(logk25, dh25, dcp, Tx)
        
    # CO32
    logk25 = 11.5
    dh25 = 0
    dcp = 0 
    trivKthermo [1,1,2] = VH2(logk25, dh25, dcp, Tx)
        
    # HCO3
    logk25 = 1.95
    dh25 = 0
    dcp = 0 
    trivKthermo [0,1,3] = VH2(logk25, dh25, dcp, Tx)
        
    # F
    logk25 = 3.28
    dh25 = 0
    dcp = 0 
    trivKthermo [0,1,4] = VH2(logk25, dh25, dcp, Tx)
        
    # OH
    logk25 = 5.6
    rad = 1.034 + 1.4
    dh25 = (-0.836 * logk25 + 5.7 * (3 / rad)) * 4184 - DH25H2O
    dcp = Cp2(rad,Tx)  
    trivKthermo [0,1,5] = VH2(logk25, dh25, dcp, Tx)
        
    ## Pr
    # Cl
    logk25 = 0.32
    dh25 = 0
    dcp = 0  
    trivKthermo [0,2,0] = VH2(logk25, dh25, dcp, Tx)
        
    # SO4
    logk25 = 3.27
    dh25 = 0
    dcp = 0  
    trivKthermo [0,2,1] = VH2(logk25, dh25, dcp, Tx)
        
    # CO3
    logk25 = 7.03
    dh25 = 0
    dcp = 0  
    trivKthermo [0,2,2] = VH2(logk25, dh25, dcp, Tx)
        
    # CO32
    logk25 = 11.65
    dh25 = 0
    dcp = 0  
    trivKthermo [1,2,2] = VH2(logk25, dh25, dcp, Tx)
        
    # HCO3
    logk25 = 1.89
    dh25 = 0
    dcp = 0  
    trivKthermo [0,2,3] = VH2(logk25, dh25, dcp, Tx)
        
    # F
    logk25 = 3.48
    dh25 = 0
    dcp = 0  
    trivKthermo [0,2,4] = VH2(logk25, dh25, dcp, Tx)
        
    # OH
    logk25 = 5.6
    rad = 1.013 + 1.4
    dh25 = (-0.836 * logk25 + 5.7 * (3 / rad)) * 4184 - DH25H2O
    dcp = Cp2(rad,Tx)  
    trivKthermo [0,2,5] = VH2(logk25, dh25, dcp, Tx)

    ## Nd
    # Cl
    logk25 = 0.32
    dh25 = 0
    dcp = 0  
    trivKthermo [0,3,0] = VH2(logk25, dh25, dcp, Tx)
        
    # SO4
    logk25 = 3.26
    dh25 = 0
    dcp = 0  
    trivKthermo [0,3,1] = VH2(logk25, dh25, dcp, Tx)
        
    # CO3
    logk25 = 7.13
    dh25 = 0
    dcp = 0  
    trivKthermo [0,3,2] = VH2(logk25, dh25, dcp, Tx)
        
    # CO32
    logk25 = 11.8
    dh25 = 0
    dcp = 0  
    trivKthermo [1,3,2] = VH2(logk25, dh25, dcp, Tx)
        
    # HCO3
    logk25 = 1.83
    dh25 = 0
    dcp = 0  
    trivKthermo [0,3,3] = VH2(logk25, dh25, dcp, Tx)
        
    # F
    logk25 = 3.56
    dh25 = 0
    dcp = 0  
    trivKthermo [0,3,4] = VH2(logk25, dh25, dcp, Tx)
        
    # OH
    logk25 = 5.67
    rad = 0.995 + 1.4
    dh25 = (-0.836 * logk25 + 5.7 * (3 / rad)) * 4184 - DH25H2O
    dcp = Cp2(rad,Tx)  
    trivKthermo [0,3,5] = VH2(logk25, dh25, dcp, Tx)
        
    ## Pm
    # Cl
    logk25 = 0.31
    dh25 = 0
    dcp = 0 
    trivKthermo [0,4,0] = VH2(logk25, dh25, dcp, Tx)
        
    # SO4
    logk25 = 3.34
    dh25 = 0
    dcp = 0
    trivKthermo [0,4,1] = VH2(logk25, dh25, dcp, Tx)
    
    # CO3
    logk25 = 7.22
    dh25 = 0
    dcp = 0 
    trivKthermo [0,4,2] = VH2(logk25, dh25, dcp, Tx)
        
    # CO32
    logk25 = 11.96
    dh25 = 0
    dcp = 0  
    trivKthermo [1,4,2] = VH2(logk25, dh25, dcp, Tx)
        
    # HCO3
    logk25 = 1.79
    dh25 = 0
    dcp = 0  
    trivKthermo [0,4,3] = VH2(logk25, dh25, dcp, Tx)
        
    # F
    logk25 = 3.63
    dh25 = 0
    dcp = 0  
    trivKthermo [0,4,4] = VH2(logk25, dh25, dcp, Tx)
        
    # OH
    logk25 = 5.77
    rad = 0.979 + 1.4
    dh25 = (-0.836 * logk25 + 5.7 * (3 / rad)) * 4184 - DH25H2O
    dcp = Cp2(rad,Tx)  
    trivKthermo [0,4,5] = VH2(logk25, dh25, dcp, Tx)

    ## Sm
    # Cl
    logk25 = 0.3
    dh25 = 0
    dcp = 0 
    trivKthermo [0,5,0] = VH2(logk25, dh25, dcp, Tx)
        
    # SO4
    logk25 = 3.28
    dh25 = 0
    dcp = 0  
    trivKthermo [0,5,1] = VH2(logk25, dh25, dcp, Tx)
        
    # CO3
    logk25 = 7.3
    dh25 = 0
    dcp = 0  
    trivKthermo [0,5,2] = VH2(logk25, dh25, dcp, Tx)
        
    # CO32
    logk25 = 12.11
    dh25 = 0
    dcp = 0  
    trivKthermo [1,5,2] = VH2(logk25, dh25, dcp, Tx)
        
    # HCO3
    logk25 = 1.75
    dh25 = 0
    dcp = 0  
    trivKthermo [0,5,3] = VH2(logk25, dh25, dcp, Tx)
        
    # F
    logk25 = 3.58
    dh25 = 0
    dcp = 0  
    trivKthermo [0,5,4] = VH2(logk25, dh25, dcp, Tx)
        
    # OH
    logk25 = 5.81
    rad = 0.964 + 1.4
    dh25 = (-0.836 * logk25 + 5.7 * (3 / rad)) * 4184 - DH25H2O
    dcp = Cp2(rad,Tx)  
    trivKthermo [0,5,5] = VH2(logk25, dh25, dcp, Tx)

    ## Eu
    # Cl
    logk25 = 0.28
    dh25 = 0
    dcp = 0  
    trivKthermo [0,6,0] = VH2(logk25, dh25, dcp, Tx)
        
    # SO4
    logk25 = 3.37
    dh25 = 0
    dcp = 0  
    trivKthermo [0,6,1] = VH2(logk25, dh25, dcp, Tx)
        
    # CO3
    logk25 = 7.37
    dh25 = 0
    dcp = 0  
    trivKthermo [0,6,2] = VH2(logk25, dh25, dcp, Tx)
        
    # CO32
    logk25 = 12.24
    dh25 = 0
    dcp = 0  
    trivKthermo [1,6,2] = VH2(logk25, dh25, dcp, Tx)
        
    # HCO3
    logk25 = 1.6
    dh25 = 0
    dcp = 0  
    trivKthermo [0,6,3] = VH2(logk25, dh25, dcp, Tx)
        
    # F
    logk25 = 3.63
    dh25 = 0
    dcp = 0  
    trivKthermo [0,6,4] = VH2(logk25, dh25, dcp, Tx)
        
    # OH
    logk25 = 5.83
    rad = 0.95 + 1.4
    dh25 = (-0.836 * logk25 + 5.7 * (3 / rad)) * 4184 - DH25H2O
    dcp = Cp2(rad,Tx)  
    trivKthermo [0,6,5] = VH2(logk25, dh25, dcp, Tx)

    ## Gd
    # Cl
    logk25 = 0.28
    dh25 = 0
    dcp = 0  
    trivKthermo [0,7,0] = VH2(logk25, dh25, dcp, Tx)
        
    # SO4
    logk25 = 3.25
    dh25 = 0
    dcp = 0  
    trivKthermo [0,7,1] = VH2(logk25, dh25, dcp, Tx)
        
    # CO3
    logk25 = 7.44
    dh25 = 0
    dcp = 0  
    trivKthermo [0,7,2] = VH2(logk25, dh25, dcp, Tx)
        
    # CO32
    logk25 = 12.39
    dh25 = 0
    dcp = 0  
    trivKthermo [1,7,2] = VH2(logk25, dh25, dcp, Tx)
        
    # HCO3
    logk25 = 1.72
    dh25 = 0
    dcp = 0  
    trivKthermo [0,7,3] = VH2(logk25, dh25, dcp, Tx)
        
    # F
    logk25 = 3.75
    dh25 = 0
    dcp = 0  
    trivKthermo [0,7,4] = VH2(logk25, dh25, dcp, Tx)
        
    # OH
    logk25 = 5.787
    rad = 0.938 + 1.4
    dh25 = (-0.836 * logk25 + 5.7 * (3 / rad)) * 4184 - DH25H2O
    dcp = Cp2(rad,Tx) 
    trivKthermo [0,7,5] = VH2(logk25, dh25, dcp, Tx)
        
    ## Tb
    # Cl
    logk25 = 0.27
    dh25 = 0
    dcp = 0  
    trivKthermo [0,8,0] = VH2(logk25, dh25, dcp, Tx)
        
    # SO4
    logk25 = 3.2
    dh25 = 0
    dcp = 0  
    trivKthermo [0,8,1] = VH2(logk25, dh25, dcp, Tx)
        
    # CO3
    logk25 = 7.5
    dh25 = 0
    dcp = 0  
    trivKthermo [0,8,2] = VH2(logk25, dh25, dcp, Tx)
        
    # CO32
    logk25 = 12.52
    dh25 = 0
    dcp = 0  
    trivKthermo [1,8,2] = VH2(logk25, dh25, dcp, Tx)
        
    # HCO3
    logk25 = 1.71
    dh25 = 0
    dcp = 0  
    trivKthermo [0,8,3] = VH2(logk25, dh25, dcp, Tx)
        
    # F
    logk25 = 3.85
    dh25 = 0
    dcp = 0  
    trivKthermo [0,8,4] = VH2(logk25, dh25, dcp, Tx)
        
    # OH
    logk25 = 5.98
    rad = 0.923 + 1.4
    dh25 = (-0.836 * logk25 + 5.7 * (3 / rad)) * 4184 - DH25H2O
    dcp = Cp2(rad,Tx)  
    trivKthermo [0,8,5] = VH2(logk25, dh25, dcp, Tx)
        
    ## Dy
    # Cl
    logk25 = 0.27
    dh25 = 0
    dcp = 0  
    trivKthermo [0,9,0] = VH2(logk25, dh25, dcp, Tx)
        
    # SO4
    logk25 = 3.15
    dh25 = 0
    dcp = 0  
    trivKthermo [0,9,1] = VH2(logk25, dh25, dcp, Tx)
        
    # CO3
    logk25 = 7.55
    dh25 = 0
    dcp = 0 
    trivKthermo [0,9,2] = VH2(logk25, dh25, dcp, Tx)
        
    # CO32
    logk25 = 12.65
    dh25 = 0
    dcp = 0  
    trivKthermo [1,9,2] = VH2(logk25, dh25, dcp, Tx)
        
    # HCO3
    logk25 = 1.72
    dh25 = 0
    dcp = 0  
    trivKthermo [0,9,3] = VH2(logk25, dh25, dcp, Tx)
        
    # F
    logk25 = 3.89
    dh25 = 0
    dcp = 0  
    trivKthermo [0,9,4] = VH2(logk25, dh25, dcp, Tx)
        
    # OH
    logk25 = 6.04
    rad = 0.908 + 1.4
    dh25 = (-0.836 * logk25 + 5.7 * (3 / rad)) * 4184 - DH25H2O
    dcp = Cp2(rad,Tx)  
    trivKthermo [0,9,5] = VH2(logk25, dh25, dcp, Tx)

    ## Ho
    # Cl
    logk25 = 0.27
    dh25 = 0
    dcp = 0  
    trivKthermo [0,10,0] = VH2(logk25, dh25, dcp, Tx)
        
    # SO4
    logk25 = 3.16
    dh25 = 0
    dcp = 0  
    trivKthermo [0,10,1] = VH2(logk25, dh25, dcp, Tx)
        
    # CO3
    logk25 = 7.59
    dh25 = 0
    dcp = 0 
    trivKthermo [0,10,2] = VH2(logk25, dh25, dcp, Tx)
        
    # CO32
    logk25 = 12.77
    dh25 = 0
    dcp = 0 
    trivKthermo [1,10,2] = VH2(logk25, dh25, dcp, Tx)
        
    # HCO3
    logk25 = 1.73
    dh25 = 0
    dcp = 0  
    trivKthermo [0,10,3] = VH2(logk25, dh25, dcp, Tx)
        
    # F
    logk25 = 3.95
    dh25 = 0
    dcp = 0  
    trivKthermo [0,10,4] = VH2(logk25, dh25, dcp, Tx)
        
    # OH
    logk25 = 6.1
    rad = 0.894 + 1.4
    dh25 = (-0.836 * logk25 + 5.7 * (3 / rad)) * 4184 - DH25H2O
    dcp = Cp2(rad,Tx)  
    trivKthermo [0,10,5] = VH2(logk25, dh25, dcp, Tx)
        
    ## Er
    # Cl
    logk25 = 0.28
    dh25 = 0
    dcp = 0  
    trivKthermo [0,11,0] = VH2(logk25, dh25, dcp, Tx)
        
    # SO4
    logk25 = 3.15
    dh25 = 0
    dcp = 0  
    trivKthermo [0,11,1] = VH2(logk25, dh25, dcp, Tx)
        
    # CO3
    logk25 = 7.63
    dh25 = 0
    dcp = 0  
    trivKthermo [0,11,2] = VH2(logk25, dh25, dcp, Tx)
        
    # CO32
    logk25 = 12.88
    dh25 = 0
    dcp = 0 
    trivKthermo [1,11,2] = VH2(logk25, dh25, dcp, Tx)
        
    # HCO3
    logk25 = 1.76
    dh25 = 0
    dcp = 0  
    trivKthermo [0,11,3] = VH2(logk25, dh25, dcp, Tx)
        
    # F 
    logk25 = 3.98
    dh25 = 0
    dcp = 0  
    trivKthermo [0,11,4] = VH2(logk25, dh25, dcp, Tx)
        
    # OH
    logk25 = 6.15
    rad = 0.881 + 1.4
    dh25 = (-0.836 * logk25 + 5.7 * (3 / rad)) * 4184 - DH25H2O
    dcp = Cp2(rad,Tx)  
    trivKthermo [0,11,5] = VH2(logk25, dh25, dcp, Tx)
        
    ## Tm
    # Cl
    logk25 = 0.27
    dh25 = 0
    dcp = 0  
    trivKthermo [0,12,0] = VH2(logk25, dh25, dcp, Tx)
        
    # SO4
    logk25 = 3.07
    dh25 = 0
    dcp = 0  
    trivKthermo [0,12,1] = VH2(logk25, dh25, dcp, Tx)
        
    # CO3
    logk25 = 7.66
    dh25 = 0
    dcp = 0 
    trivKthermo [0,12,2] = VH2(logk25, dh25, dcp, Tx)
        
    # CO32
    logk25 = 13
    dh25 = 0
    dcp = 0 
    trivKthermo [1,12,2] = VH2(logk25, dh25, dcp, Tx)
        
    # HCO3
    logk25 = 1.79
    dh25 = 0
    dcp = 0  
    trivKthermo [0,12,3] = VH2(logk25, dh25, dcp, Tx)
        
    # F
    logk25 = 3.99
    dh25 = 0
    dcp = 0  
    trivKthermo [0,12,4] = VH2(logk25, dh25, dcp, Tx)
        
    # OH
    logk25 = 6.19
    rad = 0.87 + 1.4
    dh25 = (-0.836 * logk25 + 5.7 * (3 / rad)) * 4184 - DH25H2O
    dcp = Cp2(rad,Tx)  
    trivKthermo [0,12,5] = VH2(logk25, dh25, dcp, Tx)

    ## Yb
    # Cl
    logk25 = 0.16
    dh25 = 0
    dcp = 0  
    trivKthermo [0,13,0] = VH2(logk25, dh25, dcp, Tx)
        
    # SO4
    logk25 = 3.06
    dh25 = 0
    dcp = 0  
    trivKthermo [0,13,1] = VH2(logk25, dh25, dcp, Tx)
        
    # CO3
    logk25 = 7.67
    dh25 = 0
    dcp = 0  
    trivKthermo [0,13,2] = VH2(logk25, dh25, dcp, Tx)
        
    # CO32
    logk25 = 13.08
    dh25 = 0
    dcp = 0 
    trivKthermo [1,13,2] = VH2(logk25, dh25, dcp, Tx)
        
    # HCO3
    logk25 = 1.84
    dh25 = 0
    dcp = 0 
    trivKthermo [0,13,3] = VH2(logk25, dh25, dcp, Tx)
        
    # F
    logk25 = 4.02
    dh25 = 0
    dcp = 0  
    trivKthermo [0,13,4] = VH2(logk25, dh25, dcp, Tx)
        
    # OH
    logk25 = 6.22
    rad = 0.858 + 1.4
    dh25 = (-0.836 * logk25 + 5.7 * (3 / rad)) * 4184 - DH25H2O
    dcp = Cp2(rad,Tx)  
    trivKthermo [0,13,5] = VH2(logk25, dh25, dcp, Tx)
        
    ## Lu
    # Cl
    logk25 = -0.03
    dh25 = 0
    dcp = 0  
    trivKthermo [0,14,0] = VH2(logk25, dh25, dcp, Tx)
        
    # SO4
    logk25 = 3.01
    dh25 = 0
    dcp = 0  
    trivKthermo [0,14,1] = VH2(logk25, dh25, dcp, Tx)
        
    # CO3
    logk25 = 7.7
    dh25 = 0
    dcp = 0  
    trivKthermo [0,14,2] = VH2(logk25, dh25, dcp, Tx)
        
    # CO32
    logk25 = 13.2
    dh25 = 0
    dcp = 0  
    trivKthermo [1,14,2] = VH2(logk25, dh25, dcp, Tx)
        
    # HCO3
    logk25 = 1.9
    dh25 = 0
    dcp = 0  
    trivKthermo [0,14,3] = VH2(logk25, dh25, dcp, Tx)
        
    # F
    logk25 = 4.05
    dh25 = 0
    dcp = 0  
    trivKthermo [0,14,4] = VH2(logk25, dh25, dcp, Tx)
        
    # OH
    logk25 = 6.24
    rad = 0.85 + 1.4
    dh25 = (-0.836 * logk25 + 5.7 * (3 / rad)) * 4184 - DH25H2O
    dcp = Cp2(rad,Tx)  
    trivKthermo [0,14,5] = VH2(logk25, dh25, dcp, Tx)

    ## Y
    # Cl
    logk25 = 0.8
    dh25 = 0
    dcp = 0  
    trivKthermo [0,15,0] = VH2(logk25, dh25, dcp, Tx)
        
    # SO4
    logk25 = 3.47
    dh25 = 0
    dcp = 0  
    trivKthermo [0,15,1] = VH2(logk25, dh25, dcp, Tx)
        
    # SO4 2
    logk25 = 5.3
    dh25 = 0
    dcp = 0  
    trivKthermo [1,15,1] = VH2(logk25, dh25, dcp, Tx)
        
    # CO3
    logk25 = 6.94
    dh25 = 0
    dcp = 0  
    trivKthermo [0,15,2] = VH2(logk25, dh25, dcp, Tx)
        
    # F
    logk25 = 4.8
    dh25 = 0
    dcp = 0  
    trivKthermo [0,15,4] = VH2(logk25, dh25, dcp, Tx)
        
    # F2
    logk25 = 8.74
    dh25 = 0
    dcp = 0  
    trivKthermo [1,15,4] = VH2(logk25, dh25, dcp, Tx)
        
    # F3
    logk25 = 12.01
    dh25 = 0
    dcp = 0  
    trivKthermo [2,15,4] = VH2(logk25, dh25, dcp, Tx)
        
    # OH
    logk25 = pKw - 7.7
    rad = 0.893 + 1.4
    dh25 = (-0.836 * logk25 + 5.7 * (3 / rad)) * 4184 - DH25H2O
    dcp = Cp2(rad,Tx)  
    trivKthermo [0,15,5] = VH2(logk25, dh25, dcp, Tx)
        
    # OH2
    logk25p = logk25
    dh25p = dh25
    logk25 = 2 * pKw - 16.4
    dh25 = (-1.36 * (logk25 - logk25p) - 5.3 + 3.64 * 2) * 4184 - DH25H2O + dh25p
    dcp = 0  
    trivKthermo [1,15,5] = VH2(logk25, dh25, dcp, Tx)
        
    # OH3
    logk25p = logk25
    dh25p = dh25
    logk25 = 3 * pKw - 26
    dh25 = (-1.36 * (logk25 - logk25p) - 5.3 + 3.64 * 1) * 4184 - DH25H2O + dh25p
    dcp = 0  
    trivKthermo [2,15,5] = VH2(logk25, dh25, dcp, Tx)
        
    # OH4
    logk25p = logk25
    dh25p = dh25
    logk25 = 4 * pKw - 36.5
    dh25 = (-1.36 * (logk25 - logk25p) - 5.3 + 3.64 * 0) * 4184 - DH25H2O + dh25p
    dcp = 0  
    trivKthermo [3,15,5] = VH2(logk25, dh25, dcp, Tx)
        
    ## Al
    # CO3
    logk25 = 8.43
    dh25 = 0
    dcp = 0  
    trivKthermo [0,16,2] = VH2(logk25, dh25, dcp, Tx)
        
    # F
    logk25 = 7.01
    dh25 = 0
    dcp = 0  
    trivKthermo [0,16,4] = VH2(logk25, dh25, dcp, Tx)
        
    # F2
    logk25 = 12.73
    dh25 = 0
    dcp = 0  
    trivKthermo [1,16,4] = VH2(logk25, dh25, dcp, Tx)
        
    # F3
    logk25 = 16.71
    dh25 = 0
    dcp = 0  
    trivKthermo [2,16,4] = VH2(logk25, dh25, dcp, Tx)
        
    # F4
    logk25 = 19.67
    dh25 = 0
    dcp = 0  
    trivKthermo [3,16,4] = VH2(logk25, dh25, dcp, Tx)
        
    # OH
    logk25 = pKw - 4.97
    rad = 0.51 + 1.4
    dh25 = (-0.836 * logk25 + 5.7 * (3 / rad)) * 4184 - DH25H2O
    dcp = Cp2(rad,Tx)  
    trivKthermo [0,16,5] = VH2(logk25, dh25, dcp, Tx)
        
    # OH2
    logk25p = logk25
    dh25p = dh25
    logk25 = 2 * pKw - 9.3
    dh25 = (-1.36 * (logk25 - logk25p) - 5.3 + 3.64 * 2) * 4184 - DH25H2O + dh25p
    dcp = 0  
    trivKthermo [1,16,5] = VH2(logk25, dh25, dcp, Tx)
        
    # OH3
    logk25p = logk25
    dh25p = dh25
    logk25 = 3 * pKw - 15
    dh25 = (-1.36 * (logk25 - logk25p) - 5.3 + 3.64 * 1) * 4184 - DH25H2O + dh25p
    dcp = 0  
    trivKthermo [2,16,5] = VH2(logk25, dh25, dcp, Tx)
        
    # OH4
    logk25p = logk25
    dh25p = dh25
    logk25 = 4 * pKw - 23
    dh25 = (-1.36 * (logk25 - logk25p) - 5.3 + 3.64 * 0) * 4184 - DH25H2O + dh25p
    dcp = 0  
    trivKthermo [3,16,5] = VH2(logk25, dh25, dcp, Tx)
        
    #  F5
    # lnk25 = 20.73; dh25 = 0; dcp = 0;  
    # trivKthermo (17+Y4,5,5) = VH2(lnk25, dh25, dcp, Tx)
        
    #  F6
    # lnk25 = 20.46; dh25 = 0; dcp = 0;  
    # trivKthermo (17+Y4,5,6) = VH2(lnk25, dh25, dcp, Tx)
        
    ## Ga
    # Cl
    logk25 = 0.89
    dh25 = 0
    dcp = 0  
    trivKthermo [0,17,0] = VH2(logk25, dh25, dcp, Tx)
        
    # CO3
    logk25 = 8.79
    dh25 = 0
    dcp = 0  
    trivKthermo [0,17,2] = VH2(logk25, dh25, dcp, Tx)
        
    # F
    logk25 = 5.28
    dh25 = 0
    dcp = 0  
    trivKthermo [0,17,4] = VH2(logk25, dh25, dcp, Tx)
        
    # F2
    logk25 = 9.61
    dh25 = 0
    dcp = 0 
    trivKthermo [1,17,4] = VH2(logk25, dh25, dcp, Tx)
        
    # F3
    logk25 = 12.21
    dh25 = 0
    dcp = 0  
    trivKthermo [2,17,4] = VH2(logk25, dh25, dcp, Tx)
        
    # OH
    logk25 = pKw - 2.6
    rad = 0.81 + 1.4
    dh25 = (-0.836 * logk25 + 5.7 * (3 / rad)) * 4184 - DH25H2O
    dcp = Cp2(rad,Tx)  
    trivKthermo [0,17,5] = VH2(logk25, dh25, dcp, Tx)
        
    # OH2
    logk25p = logk25
    dh25p = dh25
    logk25 = 2 * pKw - 5.9
    dh25 = (-1.36 * (logk25 - logk25p) - 5.3 + 3.64 * 2) * 4184 - DH25H2O + dh25p
    dcp = 0 
    trivKthermo [1,17,5] = VH2(logk25, dh25, dcp, Tx)
        
    # OH3
    logk25p = logk25
    dh25p = dh25
    logk25 = 3 * pKw - 10.3
    dh25 = (-1.36 * (logk25 - logk25p) - 5.3 + 3.64 * 1) * 4184 - DH25H2O + dh25p
    dcp = 0  
    trivKthermo [2,17,5] = VH2(logk25, dh25, dcp, Tx)
        
    # OH4
    logk25p = logk25
    dh25p = dh25
    logk25 = 4 * pKw - 16.6
    dh25 = (-1.36 * (logk25 - logk25p) - 5.3 + 3.64 * 0) * 4184 - DH25H2O + dh25p
    dcp = 0  
    trivKthermo [3,17,5] = VH2(logk25, dh25, dcp, Tx)
        
    ## In
    # Cl
    logk25 = 3.26
    dh25 = 0
    dcp = 0 
    trivKthermo [0,18,0] = VH2(logk25, dh25, dcp, Tx)
        
    # Cl2
    logk25 = 6.38
    dh25 = 0
    dcp = 0  
    trivKthermo [1,18,0] = VH2(logk25, dh25, dcp, Tx)
        
    # Cl3
    logk25 = 5.83
    dh25 = 0
    dcp = 0 
    trivKthermo [2,18,0] = VH2(logk25, dh25, dcp, Tx)
        
    # SO4
    logk25 = 3.86
    dh25 = 0
    dcp = 0 
    trivKthermo [0,18,1] = VH2(logk25, dh25, dcp, Tx)
        
    # SO4 2
    logk25 = 5.43
    dh25 = 0
    dcp = 0  
    trivKthermo [1,18,1] = VH2(logk25, dh25, dcp, Tx)
        
    # SO4 3
    logk25 = 5.52
    dh25 = 0
    dcp = 0  
    trivKthermo [2,18,1] = VH2(logk25, dh25, dcp, Tx)
        
    # CO3
    logk25 = 7.6
    dh25 = 0
    dcp = 0  
    trivKthermo [0,18,2] = VH2(logk25, dh25, dcp, Tx)
        
    # F
    logk25 = 4.6
    dh25 = 0
    dcp = 0 
    trivKthermo [0,18,4] = VH2(logk25, dh25, dcp, Tx)
        
    # F2
    logk25 = 8.1
    dh25 = 0
    dcp = 0  
    trivKthermo [1,18,4] = VH2(logk25, dh25, dcp, Tx)
        
    # F3
    logk25 = 10.3
    dh25 = 0
    dcp = 0  
    trivKthermo [2,18,4] = VH2(logk25, dh25, dcp, Tx)
        
    # F4
    logk25 = 11.51
    dh25 = 0
    dcp = 0  
    trivKthermo [3,18,4] = VH2(logk25, dh25, dcp, Tx)
        
    # OH
    logk25 = pKw - 4
    rad = 0.81 + 1.4
    dh25 = (-0.836 * logk25 + 5.7 * (3 / rad)) * 4184 - DH25H2O
    dcp = Cp2(rad,Tx)  
    trivKthermo [0,18,5] = VH2(logk25, dh25, dcp, Tx)
        
    # OH2
    logk25p = logk25
    dh25p = dh25
    logk25 = 2 * pKw - 7.82
    dh25 = (-1.36 * (logk25 - logk25p) - 5.3 + 3.64 * 2) * 4184 - DH25H2O + dh25p
    dcp = 0  
    trivKthermo [1,18,5] = VH2(logk25, dh25, dcp, Tx)
        
    # OH3
    logk25p = logk25
    dh25p = dh25
    logk25 = 3 * pKw - 12.4
    dh25 = (-1.36 * (logk25 - logk25p) - 5.3 + 3.64 * 1) * 4184 - DH25H2O + dh25p
    dcp = 0  
    trivKthermo [2,18,5] = VH2(logk25, dh25, dcp, Tx)
        
    # OH4
    logk25p = logk25
    dh25p = dh25
    logk25 = 4 * pKw - 22.07
    dh25 = (-1.36 * (logk25 - logk25p) - 5.3 + 3.64 * 0) * 4184 - DH25H2O + dh25p
    dcp = 0  
    trivKthermo [3,18,5] = VH2(logk25, dh25, dcp, Tx)

    ## Fe(III)
    # Cl
    logk25 = 1.28
    dh25 = 0
    dcp = 0 
    trivKthermo [0,19,0] = VH2(logk25, dh25, dcp, Tx)
        
    # Cl2
    logk25 = 1.16
    dh25 = 0
    dcp = 0  
    trivKthermo [1,19,0] = VH2(logk25, dh25, dcp, Tx)
        
    # SO4
    logk25 = 4.27
    dh25 = 0
    dcp = 0  
    trivKthermo [0,19,1] = VH2(logk25, dh25, dcp, Tx)
        
    # SO4 2
    logk25 = 6.11
    dh25 = 0
    dcp = 0  
    trivKthermo [1,19,1] = VH2(logk25, dh25, dcp, Tx)
        
    # F
    logk25 = 6.03
    dh25 = 0
    dcp = 0
    trivKthermo [0,19,4] = VH2(logk25, dh25, dcp, Tx)
        
    # F2
    logk25 = 10.66
    dh25 = 0
    dcp = 0  
    trivKthermo [1,19,4] = VH2(logk25, dh25, dcp, Tx)
        
    # F3
    logk25 = 13.66
    dh25 = 0
    dcp = 0  
    trivKthermo [2,19,4] = VH2(logk25, dh25, dcp, Tx)
        
    # OH
    trivKthermo [0,19,5]= 10**(pKw + (2.4 - 1366 / Tx))
    # OH2
    trivKthermo [1,19,5]= 10**(2 * pKw + (-0.18 - 1996.5 / Tx))
    # OH3
    trivKthermo [2,19,5]= 10**(3 * pKw + (0.53 - 4042 / Tx))
    # OH4
    trivKthermo [3,19,5]= 10**(4 * pKw + (5.26 - 7976 / Tx))
        

    ## ************************** Minor an - Major cat  *************************
    mnAn_Kthermo = np.zeros([18,5])
    # ## HSO4
    # ## HS
    # ## I
    # param_NaI = [9.8742e1, 3.2917e-2, -2.7576e+3, -4.0748e1, -4.3058e1, 0.0]; % [1.54, 7.33455] kJ/mol	Range:  0-300
    # param_KI = [1.0816e2, 3.3683e-2, -3.2143e+3, -4.4054e1, -5.0187e1, 0.0];  % [ -1.598, 9.16296] 	kJ/mol	Range:  0-300
    # ## ClO3
    # ## ClO4
    # ## BrO3
    # ## CNS
    # ## NO2
    # ## NO3
    # ## H2PO4
    # # Mg 
    # mnAn_Kthermo(10,3) = 10^1.13; 
    # # Ca
    # mnAn_Kthermo(10,4) = 10^1;
    # ## HPO4
    # # Mg
    # mnAn_Kthermo(11,3) = 10^2.7; 
    # # Ca
    # mnAn_Kthermo(11,4) = 10^2.74; 
    # ## PO4
    # # Mg
    # mnAn_Kthermo(12,3) = 10^5.63;
    # # Ca
    # mnAn_Kthermo(12,4)= 10^7.1; 
    # 
    # BMGH2ASO4 = 0; 
    # BCAH2ASO4 = 0;
    # BMGHASO4 = 7.83;
    # BCAHASO4 = 4.28
    # BMGASO4 = 55.6;
    # BCAASO4 = 1264;
    # 
    # ## H2AsO4
    # ## HAsO4
    # ## AsO4
    # ## HSO3
    # ## SO3
    # # 
    # mnAn_Kthermo(17,3) = exp(2.36);          % Roy et al. 1991 j Sol Chem
    # ## Acetate 
    
    
    
    return mjKthermo, mnCat_Kthermo, divKthermo, trivKthermo, mnAn_Kthermo

In [3]:
mjK, mnCat, divK , trivK, mnAn = Kthermo(25)
np.save('mjKthermo_T25.npy', mjK)
print(mjK)

[[1.75754254e-01 5.01187234e+00 1.86208714e+01 5.62341325e-01
  4.67339067e-02 1.06959380e-01 1.65958691e+00 6.60693448e-01]
 [3.35809321e-02 7.02642685e+00 0.00000000e+00 0.00000000e+00
  1.90239911e-02 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [7.80694708e-01 2.67043780e+02 9.54246334e+02 1.16141564e+01
  0.00000000e+00 6.60693448e+01 3.71535229e+01 1.54037108e+02]
 [2.11952224e-01 1.99526231e+02 1.67529331e+03 1.18501483e+01
  0.00000000e+00 8.70963590e+00 6.30957344e+01 1.99526231e+01]
 [5.93934636e-01 1.94984460e+02 1.00087885e+03 1.52966401e+01
  0.00000000e+00 1.46150249e+00 3.54813389e+01 6.60693448e+00]]


In [4]:
mjK, mnCat, divK , trivK, mnAn = Kthermo(25)
np.save('mnCat_Kthermo_T25.npy', mnCat)
print(mnCat)

[[[0.00000000e+00 1.37891811e-01 0.00000000e+00 0.00000000e+00
   0.00000000e+00 6.70529362e-04 0.00000000e+00]
  [3.28072669e-02 5.01187234e+00 0.00000000e+00 0.00000000e+00
   0.00000000e+00 2.04173794e+00 0.00000000e+00]
  [1.14190202e-01 8.70963590e+00 0.00000000e+00 0.00000000e+00
   6.32337296e-02 9.53049158e+00 0.00000000e+00]
  [7.58158215e-01 1.09647820e+01 5.12861384e-01 0.00000000e+00
   5.55737795e-01 0.00000000e+00 0.00000000e+00]
  [0.00000000e+00 1.28824955e+01 0.00000000e+00 0.00000000e+00
   0.00000000e+00 0.00000000e+00 0.00000000e+00]
  [3.35454587e-01 5.01187234e+02 5.16613975e+02 9.58544655e+00
   0.00000000e+00 6.88863473e-01 0.00000000e+00]
  [1.25892541e+03 0.00000000e+00 0.00000000e+00 0.00000000e+00
   0.00000000e+00 0.00000000e+00 0.00000000e+00]]

 [[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
   0.00000000e+00 0.00000000e+00 0.00000000e+00]
  [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
   0.00000000e+00 0.00000000e+00 0.0000

In [5]:
mjK, mnCat, divK , trivK, mnAn = Kthermo(25)
np.save('divKthermo_T25.npy', divK)
print(divK)

[[[2.45470892e+00 1.81970086e+02 1.25892541e+04 1.90546072e+01
   0.00000000e+00 2.51188643e+03]
  [2.08929613e+00 1.58489319e+02 2.81838293e+05 2.95120923e+01
   0.00000000e+00 3.09029543e+04]
  [1.81970086e+00 1.14815362e+03 3.09029543e+04 2.57039578e+01
   0.00000000e+00 2.18776162e+04]
  [1.47910839e+00 1.94984460e+02 2.34422882e+05 4.26579519e+01
   0.00000000e+00 1.34896288e+04]
  [3.38844156e+00 2.29086765e+02 5.37031796e+06 6.60693448e+01
   0.00000000e+00 1.00000000e+06]
  [2.13796209e+00 2.29086765e+02 5.12861384e+04 5.49540874e+01
   0.00000000e+00 1.07151931e+05]
  [0.00000000e+00 0.00000000e+00 6.30957344e+10 0.00000000e+00
   6.30957344e+06 1.31825674e+08]
  [0.00000000e+00 0.00000000e+00 3.38844156e+07 0.00000000e+00
   4.07380278e+05 3.98108087e+08]
  [3.63078055e+01 2.81838293e+02 2.23872114e+04 2.34422882e+01
   0.00000000e+00 8.12830516e+03]
  [3.01995172e+01 5.62341325e+02 1.00000000e+07 1.12201845e+02
   0.00000000e+00 1.90546072e+06]]

 [[0.00000000e+00 0.00000000

In [6]:
mjK, mnCat, divK , trivK, mnAn = Kthermo(25)
np.save('trivKthermo_T25.npy', trivK)
print(trivK)

[[[1.94984460e+00 1.62181010e+03 6.60693448e+06 1.04712855e+02
   1.31825674e+03 1.25892541e+05]
  [2.04173794e+00 1.94984460e+03 8.91250938e+06 8.91250938e+01
   1.90546072e+03 3.98107171e+05]
  [2.08929613e+00 1.86208714e+03 1.07151931e+07 7.76247117e+01
   3.01995172e+03 3.98107171e+05]
  [2.08929613e+00 1.81970086e+03 1.34896288e+07 6.76082975e+01
   3.63078055e+03 4.67735141e+05]
  [2.04173794e+00 2.18776162e+03 1.65958691e+07 6.16595002e+01
   4.26579519e+03 5.88843655e+05]
  [1.99526231e+00 1.90546072e+03 1.99526231e+07 5.62341325e+01
   3.80189396e+03 6.45654229e+05]
  [1.90546072e+00 2.34422882e+03 2.34422882e+07 3.98107171e+01
   4.26579519e+03 6.76082975e+05]
  [1.90546072e+00 1.77827941e+03 2.75422870e+07 5.24807460e+01
   5.62341325e+03 6.12350392e+05]
  [1.86208714e+00 1.58489319e+03 3.16227766e+07 5.12861384e+01
   7.07945784e+03 9.54992586e+05]
  [1.86208714e+00 1.41253754e+03 3.54813389e+07 5.24807460e+01
   7.76247117e+03 1.09647820e+06]
  [1.86208714e+00 1.44543977e+

In [7]:
mjK, mnCat, divK , trivK, mnAn = Kthermo(25)
np.save('mnAn_Kthermo_T25.npy', mnAn)
print(mnAn)

[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]
